In [1]:
import pandas as pd
import os
import torch
from torch.utils import data as td
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

In [2]:
DATA_DIR = './database'
filename = 'MultiTox.csv'

## Get some data from overall dataset

In [33]:
def conformer_choice(props):
    probabilities=[props[key]['energy'] for key in props.keys()]
    conformer=np.random.choice(range(len(props)),1,probabilities)
    return np.asscalar(conformer)

In [5]:
THRESH_NAN = 3

In [46]:
data = pd.read_csv(os.path.join(DATA_DIR,filename))

In [27]:
columns = list(data)
columns.remove('SMILES')

In [8]:
nan = False
for i in range(int(len(data)/10)):
    data_selected = data.iloc[:i*10]
    for column in columns:
        if (~data_selected[column].isna()).sum()<THRESH_NAN:
            nan = True
            break
    if nan: 
        nan = False
        continue
    else:
        break

In [9]:
print('min not Nan values\t', THRESH_NAN)
print('dataset length\t\t', len(data_selected))
for column in columns:
    print(column,'\t\t', (~data_selected[column].isna()).sum())

min not Nan values	 3
dataset length		 1480
o_mus_ipr_LD 		 576
o_rat_orl_TDLo 		 19
o_mus_ipr_LDLo 		 37
o_mus_orl_TDLo 		 19
o_rat_ipr_TDLo 		 22
o_mus_ivn_LD 		 265
o_rat_ipr_LD 		 74
o_mus_orl_LD 		 536
o_mus_unr_LD 		 27
o_rat_unr_LD 		 5
o_mus_scu_LDLo 		 4
o_rat_scu_LD 		 26
o_mus_scu_LD 		 81
o_rat_ipr_LDLo 		 17
o_mus_ipr_TDLo 		 22
o_rbt_skn_LD 		 40
o_rat_orl_LD 		 168
o_rat_ivn_TDLo 		 6
o_rat_orl_LDLo 		 21
o_rbt_orl_LD 		 11
o_rbt_ivn_LD 		 3
o_rat_ivn_LD 		 23
o_mus_orl_LDLo 		 22
o_rat_skn_LD 		 15
o_mam_unr_LD 		 9
o_gpg_orl_LD 		 14
o_wmn_orl_TDLo 		 9
o_man_orl_TDLo 		 3
o_rat_scu_TDLo 		 5


In [4]:
import numpy as np
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import rdMolTransforms as rdmt

In [39]:
#number of conformers created for every molecule
NUM_CONFS=100

#amount of chemical elements taking into account
AMOUNT_OF_ELEM=6

In [40]:
def create_element_dict(data,amount=9,treshold=10, add_H=False):
    elements={}
    norm=0
    for smile in data['SMILES']:
        molecule=Chem.MolFromSmiles(smile)
        molecule=Chem.AddHs(molecule)

        for i in range(molecule.GetNumAtoms()):
            atom = molecule.GetAtomWithIdx(i)
            element=atom.GetSymbol()
            norm+=1
            if element in elements.keys():
                elements[element]+=1
            else:
                elements[element]=1
    for key in elements.keys():
        elements[key]/=norm
    from collections import OrderedDict
    dd = OrderedDict(sorted(elements.items(), key=lambda x: x[1]))
    elements=list(dd.keys())[-amount:]  
    elements=dict((elem,i) for i, elem in enumerate(elements))  
    if not add_H:
        del elements['H']
    return elements

In [41]:
elements=create_element_dict(data,amount=AMOUNT_OF_ELEM,add_H=True)

In [42]:
elements

{'Cl': 0, 'S': 1, 'N': 2, 'O': 3, 'C': 4, 'H': 5}

In [9]:
import load_data_multitox as ld
from sklearn.preprocessing import StandardScaler

In [10]:
DATASET_PATH='./'

In [11]:
conf_calc = ld.reading_sql_database(DATASET_PATH)

In [47]:
scaler = StandardScaler()
data[columns]=scaler.fit_transform(data[columns])

In [48]:
indexing, label_dict = ld.indexing_label_dict(data, conf_calc)

In [73]:
class Gauss_dataset(td.Dataset):
    def __init__(self,conf_calc,label_dict,elements,indexing, indexes,dx=0.5,dim=70):
        self.Xs=conf_calc
        self.Ys=label_dict
        self.elements=elements
        self.indexing = indexing
        self.dx = dx
#         self.sigma=sigma
#         self.dim=dim
#         self.dx=dx
#         self.kern_dim=kern_dim
        self.indexes=indexes
        self.dim = dim

    def __len__(self):
        'Denotes the total number of samples'
        return len(self.indexes)

    def __getitem__(self, index):
        from math import floor
        'Generates one sample of data'
        dimelem = len(self.elements)
        
        cube=torch.zeros((dimelem,self.dim,self.dim,self.dim))
        
        i=self.indexes[index]
        smiles=self.indexing[i]
        
        y= self.Ys[smiles]

        description=self.Xs[smiles][conformer_choice(self.Xs[smiles])]['coordinates']
#         X = gaussian_blur(description,self.elements,sigma=self.sigma,dimx=self.dim,dx=self.dx,kern_dim=self.kern_dim)

        for atom in description.keys():
        
            num_atom=elements[atom]

            for x0,y0,z0 in description[atom]:
                cube[num_atom, floor(x0/self.dx), floor(y0/self.dx), floor(z0/self.dx)]=1
        X= cube
        return X, y
    

In [64]:
def gaussian_blur(molecule,elements,sigma=2,dimx=70,dx=0.5,kern_dim=50):
    from math import floor
    
    dimelem=len(elements)
    cube=torch.zeros((dimelem,dimx,dimx,dimx))
    
    #build the kernel
    x = torch.arange(-kern_dim/4,kern_dim/4,dx)
    y = torch.arange(-kern_dim/4,kern_dim/4,dx)
    z = torch.arange(-kern_dim/4,kern_dim/4,dx)
    xx, yy, zz = torch.meshgrid((x,y,z))
    kernel = torch.exp(-(xx**2 + yy**2 + zz**2)/(2*sigma**2))
    
    for atom in molecule.keys():
        
        num_atom=elements[atom]
        
        for x0,y0,z0 in molecule[atom]:
            
            x_range=[max(floor(x[0]/dx+x0/dx+dimx/2),0),min(floor(x[-1]/dx+x0/dx+dimx/2+1),cube.shape[1])]
            y_range=[max(floor(y[0]/dx+y0/dx+dimx/2),0),min(floor(y[-1]/dx+y0/dx+dimx/2+1),cube.shape[2])]
            z_range=[max(floor(z[0]/dx+z0/dx+dimx/2),0),min(floor(z[-1]/dx+z0/dx+dimx/2+1),cube.shape[3])]
            coord_ranges=[x_range,y_range,z_range]
            for i in range(3):
                if coord_ranges[i][1]-coord_ranges[i][0]>50:
                    coord_ranges[i][1]=coord_ranges[i][0]+50
            cube_part=cube[num_atom,coord_ranges[0][0]:coord_ranges[0][1],
                           coord_ranges[1][0]:coord_ranges[1][1],
                           coord_ranges[2][0]:coord_ranges[2][1]]
            
            kern_ranges=[[],[],[]]
            for i in range(3):
                if coord_ranges[i][0]==0:
                    kern_ranges[i].append(kern_dim-cube_part.shape[i])
                else:
                    kern_ranges[i].append(0)
                if coord_ranges[i][1]==cube.shape[i+1]:
                    kern_ranges[i].append(cube_part.shape[i])
                else:
                    kern_ranges[i].append(kern_dim)
                    
            cube_part=cube_part+kernel[kern_ranges[0][0]:kern_ranges[0][1],
                                       kern_ranges[1][0]:kern_ranges[1][1],
                                       kern_ranges[2][0]:kern_ranges[2][1]]
            
            cube[num_atom,coord_ranges[0][0]:coord_ranges[0][1],
                 coord_ranges[1][0]:coord_ranges[1][1],
                 coord_ranges[2][0]:coord_ranges[2][1]]=cube_part

    return cube


In [204]:
class Net(nn.Module):
    def __init__(self, dim=70, dx=0.5,kern_dim=50,num_elems=6, num_targets=12, batch_size=args['BATCH_SIZE'], elements=elements):
        super(Net, self).__init__()

        self.sigma = Variable(torch.ones(num_elems),requires_grad=True)
        
        # initialize dimensions
        self.dim = dim
        self.num_elems = num_elems
        self.num_targets = num_targets
        self.batch_size = batch_size
        self.elements=elements
        self.dx=dx
        self.kern_dim=kern_dim

        # create layers
        self.conv1 = nn.Conv3d(num_elems, 32, kernel_size=(3, 3, 3))
        self.pool1 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3))
        self.pool2 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3))
        self.pool3 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.conv4 = nn.Conv3d(128, 256, kernel_size=(3, 3, 3))
        self.pool4 = nn.MaxPool3d(kernel_size=(2, 2, 2))
        self.fc1 = nn.Linear(2048, 1024)
        self.fc2 = nn.Linear(1024, TARGET_NUM)

        # initialize dense layer's weights
        nn.init.xavier_uniform_(self.fc1.weight)
        self.fc1.bias.data.fill_(0.01)

        self.convolution = nn.Sequential(
            self.conv1,
            self.pool1,
            nn.ReLU(),
            self.conv2,
            self.pool2,
            nn.ReLU(),
            self.conv3,
            self.pool3,
            nn.ReLU(),
            self.conv4,
            self.pool4,
            nn.ReLU()
        )

        def weights_init(m):
            if type(m) == nn.Conv3d:
                nn.init.xavier_uniform_(m.weight)
                m.bias.data.fill_(0.01)

        # initialize convolutional layers' weights
        self.convolution.apply(weights_init)
        
    def gaussian_blur (self, batch):#(molecule,elements,sigma=2,dimx=70,dx=0.5,kern_dim=50):
        from math import floor

        dimx=self.dim
        dx=self.dx
        kern_dim=self.kern_dim
        
        

        dimelem=len(elements)
#         cube=torch.zeros((dimelem,dimx,dimx,dimx))
        cube=torch.zeros(batch.shape)

        #build the kernel
        x = torch.arange(-kern_dim/4,kern_dim/4,dx)
        y = torch.arange(-kern_dim/4,kern_dim/4,dx)
        z = torch.arange(-kern_dim/4,kern_dim/4,dx)
        xx, yy, zz = torch.meshgrid((x,y,z))
#         kernel = torch.exp(-(xx**2 + yy**2 + zz**2)/(2*self.sigma**2))

#         for atom in molecule.keys():

#             num_atom=elements[atom]
        for idx,molecule in enumerate(batch):
            for num_atom in range(len(elements)):
    #             print(xx.shape)
    #             print(self.sigma.shape)
                kernel = torch.exp(-(xx**2 + yy**2 + zz**2)/(2*self.sigma[num_atom]**2))
#                 print(kernel)
                
                for x0,y0,z0 in molecule[num_atom].nonzero():

                    x_range=[max(floor(x[0]/dx+x0+dimx/2),0),min(floor(x[-1]/dx+x0+dimx/2+1),cube.shape[1])]
                    y_range=[max(floor(y[0]/dx+y0+dimx/2),0),min(floor(y[-1]/dx+y0+dimx/2+1),cube.shape[2])]
                    z_range=[max(floor(z[0]/dx+z0+dimx/2),0),min(floor(z[-1]/dx+z0+dimx/2+1),cube.shape[3])]
                    coord_ranges=[x_range,y_range,z_range]
                    for i in range(3):
                        if coord_ranges[i][1]-coord_ranges[i][0]>50:
                            coord_ranges[i][1]=coord_ranges[i][0]+50
                    cube_part=cube[idx,num_atom,coord_ranges[0][0]:coord_ranges[0][1],
                                   coord_ranges[1][0]:coord_ranges[1][1],
                                   coord_ranges[2][0]:coord_ranges[2][1]]

                    kern_ranges=[[],[],[]]
                    for i in range(3):
                        if coord_ranges[i][0]==0:
                            kern_ranges[i].append(kern_dim-cube_part.shape[i])
                        else:
                            kern_ranges[i].append(0)
                        if coord_ranges[i][1]==cube.shape[i+1]:
                            kern_ranges[i].append(cube_part.shape[i])
                        else:
                            kern_ranges[i].append(kern_dim)
                            
                    new_kern = kernel[kern_ranges[0][0]:kern_ranges[0][1],
                                               kern_ranges[1][0]:kern_ranges[1][1],
                                               kern_ranges[2][0]:kern_ranges[2][1]]*1.0
                    f = new_kern.sum()
                    f.backward()
                    print(self.sigma.grad)

                    cube_part=cube_part+kernel[kern_ranges[0][0]:kern_ranges[0][1],
                                               kern_ranges[1][0]:kern_ranges[1][1],
                                               kern_ranges[2][0]:kern_ranges[2][1]]
                    

                    cube[idx,num_atom,coord_ranges[0][0]:coord_ranges[0][1],
                         coord_ranges[1][0]:coord_ranges[1][1],
                         coord_ranges[2][0]:coord_ranges[2][1]]=cube_part
        
        return cube


    def forward(self, x):
        x_gauss = self.gaussian_blur(x)
        x_conv = self.convolution(x)
        x_vect = x_conv.view(x.shape[0], -1)
        y1 = F.relu(self.fc1(x_vect))
        y2=self.fc2(y1)
        #         multi-label (not multi-class!) classification => sigmoid non-linearity
        return y2

In [205]:
VOXEL_DIM = 70
TARGET_NUM = 29
args={'BATCH_SIZE':1}

In [206]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dim=VOXEL_DIM, num_elems=AMOUNT_OF_ELEM, num_targets=TARGET_NUM, batch_size=args['BATCH_SIZE'])
print(model)
model=model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Net(
  (conv1): Conv3d(6, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool1): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool2): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv3): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool3): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (conv4): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(1, 1, 1))
  (pool4): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=2048, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=29, bias=True)
  (convolution): Sequential(
    (0): Conv3d(6, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1))
    (1): MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2), padding=0, dilation=1, ceil_mode=False

In [207]:
train_set = Gauss_dataset(conf_calc, label_dict, elements, indexing, range(0,1000))
train_generator = td.DataLoader(train_set, batch_size=args['BATCH_SIZE'], shuffle=True)

In [208]:
for batch_idx, (data_inp, target) in enumerate(train_generator):
        data_inp = data_inp.to(device)
        target = target.to(device)
        # set gradients to zero
        optimizer.zero_grad()
        output = model(data_inp)
        mask = (target == target)
        output_masked = torch.masked_select(output, mask).type_as(output)
        target_masked = torch.masked_select(target, mask).type_as(output)
        criterion=nn.MSELoss()
        loss = criterion(output_masked, target_masked)
        loss.backward()
        optimizer.step()
#         print(model.sigma)

tensor([0., 0., 0., 0., 0., 0.])


RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.

In [172]:
model.parameters()

<generator object Module.parameters at 0x7fa551c7eca8>

In [173]:
for f in model.parameters():
    print(f)
#     print('data is')
#     print(f.data)
#     print('grad is')
#     print(f.grad)

Parameter containing:
tensor([[[[[ 5.1371e-02,  3.8120e-02,  5.9627e-02],
           [-5.5404e-02, -1.4286e-02, -9.0682e-03],
           [-6.1930e-03,  4.4690e-02,  8.6186e-03]],

          [[ 6.2113e-02, -1.7580e-02, -7.3847e-02],
           [ 6.9574e-02,  3.9660e-02, -7.1597e-02],
           [-1.9775e-02, -5.6930e-02,  3.1993e-02]],

          [[ 6.2652e-02, -4.4029e-02, -6.2310e-02],
           [-5.3702e-03, -6.9392e-02,  2.4187e-02],
           [ 7.1772e-02, -2.4553e-02, -3.8751e-02]]],


         [[[-3.4688e-03,  2.0741e-02,  6.1599e-02],
           [ 2.7281e-02, -4.8626e-02, -5.5101e-02],
           [ 2.1256e-02,  2.6283e-02, -7.0569e-02]],

          [[-5.9242e-02,  6.8024e-02,  5.2904e-03],
           [ 3.1777e-02, -3.3375e-02, -3.7471e-02],
           [ 7.9789e-03,  4.7791e-02,  4.6652e-03]],

          [[-7.0653e-03, -5.3583e-02,  6.6849e-02],
           [-9.3354e-03,  9.9189e-03, -4.0019e-02],
           [-1.7245e-02,  6.1240e-02, -1.1306e-03]]],


         [[[ 3.1290e-03,  

In [174]:
model.sigma

tensor([1., 1., 1., 1., 1., 1.], requires_grad=True)

In [75]:
data_inp,target = train_set[0]

In [87]:
data_inp.shape

torch.Size([6, 70, 70, 70])

In [96]:
data_inp[5].nonzero()

tensor([[ 1,  2,  2],
        [ 5, 67, 66],
        [ 7,  0, 65],
        [ 9, 67,  0],
        [ 9, 69,  2],
        [10,  1, 69],
        [59,  0, 69],
        [61, 66,  0],
        [63,  4, 69],
        [66, 65,  0],
        [67,  4, 69]])

In [57]:
elements

{'Cl': 0, 'S': 1, 'N': 2, 'O': 3, 'C': 4, 'H': 5}

In [126]:
model(data_inp)

AttributeError: 'Tensor' object has no attribute 'apply'